In [ ]:
# Just bring in some extra libraries

import numpy as np
import matplotlib.pylab as plt

%matplotlib notebook

from scipy.interpolate import interp1d

In [ ]:
# Here is where the data lives
data_directory = "./data"

In [ ]:
# Get the labels
labelfilename = "%s/ja_labels.csv" % (data_directory)
labelfile = open(labelfilename,"r")

labels = labelfile.readline()

labels = labels.split(',')

#'''
print(labels)

for i in range(0,len(labels)):
    print("%2d %s " % (i,labels[i]))
#''';

In [ ]:
# Get the start and stop parts
stridefilename = "%s/strides_smooth.csv" % (data_directory)
stridevals = np.loadtxt(stridefilename,skiprows=1,unpack=True,dtype=bytes,delimiter=',')

filenames = stridevals[0].astype(str)

# DO THIS FOR THE LATEST STUFF
for i in range(len(filenames)):
    filenames[i] = filenames[i].replace('ja2','ja4')

# Subtract 1 from these because the values in the file assume that the frames start at 1, not 0
firstdigitized = stridevals[1].astype(int) - 1
rstart = stridevals[2].astype(int) - 1
rend = stridevals[3].astype(int) - 1

In [ ]:
filenames

In [ ]:
# Get the start and stop parts
roughstridefilename = "%s/strides_rough.csv" % (data_directory)
roughstridevals = np.loadtxt(roughstridefilename,skiprows=1,unpack=True,dtype=bytes,delimiter=',')

roughfilenames = roughstridevals[0].astype(str)

# DO THIS FOR THE LATEST STUFF
for i in range(len(roughfilenames)):
    roughfilenames[i] = roughfilenames[i].replace('ja2','ja4')

#print(roughstridevals[1].astype(int))


# Subtract 1 from these because the values in the file assume that the frames start at 1, not 0
roughfirstdigitized = roughstridevals[1].astype(int) - 1
rstartrough = roughstridevals[2].astype(int) - 1
rendrough = roughstridevals[3].astype(int) - 1

In [ ]:
roughfilenames

In [ ]:
uniquefilenames = np.unique(filenames)
#print(filenames)
print(uniquefilenames)
print(len(uniquefilenames))

In [ ]:
# Make a master dictionary with all the data

alldata = {}
for i,label in enumerate(labels):
    print(label)
    alldata[label] = {}
    
    # Do the smooth data
    for ufilename in uniquefilenames:
        print(ufilename)
        
        #jafilename = "%s/JointAnglesFiles/%s" % (data_directory,ufilename)
        
        # For the new data
        #jafilename = "%s/JointAnglesFiles3/%s" % (data_directory,ufilename)
        jafilename = "%s/jointangles4/%s" % (data_directory,ufilename)


        javals = np.loadtxt(jafilename,skiprows=0,unpack=True,dtype=bytes,delimiter=',')

        data = {}
        for key,val in zip(labels,javals):
            data[key] = val.astype(float)

        
        index = filenames==ufilename

        # Loop over the strides
        istride = 0
        for offset,start,end in zip(firstdigitized[index],rstart[index],rend[index]):

            header = "%s:%03d:%s" % (ufilename,istride,"smooth")
            alldata[label][header] = []

            s = start
            e = end
            x = data[label][s:e]

            alldata[label][header] = x
            
            istride += 1

    # Now do the rough data
    for ufilename in uniquefilenames:
        
        #jafilename = "%s/JointAnglesFiles/%s" % (data_directory,ufilename)
        #jafilename = "%s/JointAnglesFiles3/%s" % (data_directory,ufilename)
        jafilename = "%s/jointangles4/%s" % (data_directory,ufilename)


        javals = np.loadtxt(jafilename,skiprows=0,unpack=True,dtype=bytes,delimiter=',')

        data = {}
        for key,val in zip(labels,javals):
            data[key] = val.astype(float)


        index = roughfilenames==ufilename

        istride = 0
        for offset,start,end in zip(roughfirstdigitized[index],rstartrough[index],rendrough[index]):

            header = "%s:%03d:%s" % (ufilename,istride,"rough")
            alldata[label][header] = []

            s = start
            e = end
            x = data[label][s:e]

            alldata[label][header] = x
            
            istride += 1


 

In [ ]:
#  Shoulderang, sA_v, LHipAngle, RHipAngle, Hipang, hA_v
neg = ['Shoulderang', 'sA_v', 'LHipAngle', 'RHipAngle', 'Hipang', 'hA_v']

#print(alldata.keys())

for key in neg:# alldata.keys():
    print("Making %s negative" % (key))
    for tkey in alldata[key].keys():
        alldata[key][tkey] *= -1


In [ ]:
keys = alldata[labels[0]].keys()

In [ ]:

#ksort

In [ ]:
# Write it out to files

for label in labels:
    print(label)
    keys = alldata[label].keys()
    maxval = -1
    nkeys = len(keys)
    
    
    keys = np.sort(list(keys))

    # Find the person/trial/stride with the most frames
    for key in keys:
        data = alldata[label][key]
        nvals = len(data)
        if nvals>maxval:
            maxval = nvals
    
    print(keys)
    # Write it out
    output = ",".join(keys)
    output += "\n"
    for i in range(0,maxval):
        for j,key in enumerate(keys):
            nvalskey = len(alldata[label][key])
            if i>=nvalskey:
                output += " " % ()
            else:
                output += "%s" % (alldata[label][key][i])
            
            if j<nkeys-1:
                output += ","
        
        output += "\n"
    outfilename = "variable_files/%s.csv" % (label)
    outfile = open(outfilename,"w")
    outfile.write(output)
    outfile.close()
    #print(output)

In [ ]:
plt.figure()

data = alldata['RArmAdd']

keys = data.keys()
keys = np.sort(list(keys))

for key in keys:
    
    y = data[key]
    
    if "smooth" in key:# and '5_01' in key:
        plt.plot(y,'.',alpha=0.05,color='r')
        #plt.plot(y,'.',alpha=0.5,label=key)

    elif "rough" in key:
        plt.plot(y,'.',alpha=0.2,color='k')
    
#plt.legend()
#plt.xlabel('Person',fontsize=18)
#plt.ylabel('Right elbow angle (degrees)',fontsize=18)
#plt.ylim(0,180)

    


In [ ]:
#vals = np.loadtxt('./data/JointAnglesFiles/5_01_xyzpts_ja2.txt',unpack=True,delimiter=',',dtype=bytes)
vals = np.loadtxt('./data/jointangles4/5_01_xyzpts_ja4.txt',unpack=True,delimiter=',',dtype=bytes)

y = vals[13].astype(float)

plt.figure()
plt.plot(y)

In [ ]:
y = data['1_01_xyzpts_ja4.txt:000:smooth']
plt.figure()
plt.plot(x,'.')

In [ ]:
#'''
plt.figure(figsize=(9,100))

interpdata = {}

# Loop over the variables
for k,label in enumerate(labels):
    data = alldata[label]
    interpdata[label] = {}
    
    keys = data.keys()
    keys = np.sort(list(keys))

    # Loop over the persons/trials
    lo = 1e6
    hi = -1e6
    for key in keys:
        y = data[key]
        x = np.linspace(0,1,len(y))
        interpfunc = interp1d(x, y)

        xnew = np.linspace(0, 1, num=100, endpoint=True)
        ynew = interpfunc(xnew)
        
        interpdata[label][key] = ynew
        
        if max(ynew)>hi:
            hi = max(ynew)
        if min(ynew)<lo:
            lo = min(ynew)
        
        #plt.plot(x, y,'.')
        if 'rough' in key:
            plt.subplot(41,2,2*k+2)
            plt.plot(xnew,ynew,'-',alpha=0.1,color='red')
        elif 'smooth' in key:
            plt.subplot(41,2,2*k+1)
            plt.plot(xnew,ynew,'-',alpha=0.1,color='black')
            plt.xlabel(label)

        plt.subplot(41,2,2*k+1)
        plt.ylim(lo,hi)
        plt.subplot(41,2,2*k+2)
        plt.ylim(lo,hi)

        
        
plt.tight_layout()
#'''

#plt.legend(['data', 'linear', 'cubic'], loc='best')
#>>> plt.show()

In [ ]:
# Write out the interpolation out to files

for label in labels:
    print(label)
    keys = interpdata[label].keys()
    # This is to order the keys better###################
    keys = list(keys)
    keys.sort()
    x = np.array(keys)
    indxs = np.array(['smooth' in a for a in x],dtype=bool)
    indxr = np.array(['rough' in a for a in x],dtype=bool)

    keys = []
    keys = x[indxs].tolist() + x[indxr].tolist()
    
    keys = np.sort(list(keys))
    #########################################################
    
    maxval = -1
    nkeys = len(keys)
    
    # Find the person/trial/stride with the most frames
    for key in keys:
        data = interpdata[label][key]
        nvals = len(data)
        if nvals>maxval:
            maxval = nvals

    # Write it out
    output = ",".join(keys)
    output += "\n"
    for i in range(0,maxval):
        for j,key in enumerate(keys):
            nvalskey = len(interpdata[label][key])
            if i>=nvalskey:
                output += " " % ()
            else:
                output += "%s" % (interpdata[label][key][i])
            
            if j<nkeys-1:
                output += ","
        
        output += "\n"
    outfilename = "variable_files_interpolated/%s.csv" % (label)
    outfile = open(outfilename,"w")
    outfile.write(output)
    outfile.close()
    #print(output)

In [ ]:
# Write out the max, min, mean, std dev, from the 
# interpolation out to files

for label in labels:
    print(label)
    keys = interpdata[label].keys()
    # This is to order the keys better###################
    keys = list(keys)
    keys.sort()
    x = np.array(keys)
    indxs = np.array(['smooth' in a for a in x],dtype=bool)
    indxr = np.array(['rough' in a for a in x],dtype=bool)

    keys = []
    keys = x[indxs].tolist() + x[indxr].tolist()
    
    #########################################################
    
    maxval = -1
    nkeys = len(keys)
    
    '''
    # Find the person/trial/stride with the most frames
    for key in keys:
        data = interpdata[label][key]
        nvals = len(data)
        if nvals>maxval:
            maxval = nvals
    '''
    
    # Write it out
    output = ",".join(keys)
    output += "\n"

    for i in range(0,4):
        for j,key in enumerate(keys):
    
            data = interpdata[label][key]
            
            if i==0:
                output += "%f" % (np.max(data))
            elif i==1:
                output += "%f" % (np.min(data))
            elif i==2:
                output += "%f" % (np.mean(data))
            elif i==3:
                output += "%f" % (np.std(data))
            
            if j<nkeys-1:
                output += ","
        
        output += "\n"

        
    
    '''
    for i in range(0,maxval):
        for j,key in enumerate(keys):
            nvalskey = len(interpdata[label][key])
            if i>=nvalskey:
                output += " " % ()
            else:
                output += "%s" % (interpdata[label][key][i])
            
            if j<nkeys-1:
                output += ","
        
        output += "\n"
    '''
    outfilename = "variable_files_interpolated_summary_stats/%s.csv" % (label)
    outfile = open(outfilename,"w")
    outfile.write(output)
    outfile.close()
    #print(output)

In [ ]:
data = interpdata['Rhand_v X']
#data

In [ ]:
for key in data.keys():
    x = data[key]

    if max(x)>20:
        print(key,max(x))
    

In [ ]:
# Loop over the variables

# Print out a bunch of files and save them

#'''
for k,label in enumerate(labels):
    
    plt.figure(figsize=(9,10))

    data = interpdata[label]
    
    keys = data.keys()
    keys = np.sort(list(keys))

    # Loop over the persons/trials
    lo = 1e6
    hi = -1e6
    for key in keys:
        
        person = int(key.split('_')[0])
        #print("person: ",person)
        
        y = data[key]
        x = np.linspace(0,1,len(y))
        interpfunc = interp1d(x, y)

        xnew = np.linspace(0, 1, num=100, endpoint=True)
        ynew = interpfunc(xnew)
        
        interpdata[label][key] = ynew
        
        if max(ynew)>hi:
            hi = max(ynew)
        if min(ynew)<lo:
            lo = min(ynew)
        
        #plt.plot(x, y,'.')
        if 'smooth' in key:
            plt.subplot(3,6,2*(person-1)+1)
            plt.plot(xnew,ynew,'-',alpha=0.1,color='red')
        elif 'rough' in key:
            plt.subplot(3,6,2*(person-1)+2)
            plt.plot(xnew,ynew,'-',alpha=0.1,color='black')
            #plt.xlabel(label)
    
    for i in range(18):
        plt.subplot(3,6,i+1)
        plt.ylim(lo,hi)
        name = "P%d " % (np.floor(i/2)+1)
        if i%2==0:
            name += "SMOOTH"
        else:
            name += "ROUGH"
        plt.xlabel(name)
        
    plt.gcf().suptitle(label,fontsize=24)
            
    plt.tight_layout()
    
    name = "plots/%s.pdf" % (label)
    plt.savefig(name,transparent=False)
#''';

In [ ]:
data = interpdata['RHipAngle']

plt.figure()
for key in data.keys():
    
    person = int(key.split('_')[0])
    
    if person==7:
        if sum(data[key][5:10])>0:
            plt.plot(data[key],label=key)
        
plt.legend()
        


In [ ]:
# Fill a dictionary with the extrema

extrema = {}

for label in labels:

    #vals = [[] for x in range(9)]
    
    extrema[label] = {}
    extrema[label]['max'] = {}
    extrema[label]['max']['smooth'] = [[] for x in range(9)]
    extrema[label]['max']['rough'] = [[] for x in range(9)]


    extrema[label]['min'] = {}
    extrema[label]['min']['smooth'] = [[] for x in range(9)]
    extrema[label]['min']['rough'] = [[] for x in range(9)]
    

    for key in interpdata[label].keys():
    
        person = int(key.split('_')[0]) - 1
        
        data = interpdata[label][key]

        hi = max(data)
        hiidx = data.tolist().index(hi)
        
        lo = min(data)
        loidx = data.tolist().index(lo)
        
        #print(person)
        if 'smooth' in key:
            #print("smooth: ",key)
            extrema[label]['max']['smooth'][person].append((hi,hiidx))
            extrema[label]['min']['smooth'][person].append((lo,loidx))
            
        elif 'rough' in key:
            #print("rough:  ",key)
            extrema[label]['max']['rough'][person].append((hi,hiidx))
            extrema[label]['min']['rough'][person].append((lo,loidx))            

In [ ]:
#label = 'RHipAngle'
# Make the plots for max min and write them out to files
for label in labels:
    data = extrema[label]

    hivals = [[],[]]
    hiidx = [[],[]]

    lovals = [[],[]]
    loidx = [[],[]]
    
    nstrides = [[],[]]

    plt.figure(figsize=(8,10))
    for i in range(0,9):
        

        for j,key in enumerate(['smooth','rough']):

            nstrides[j].append(len(data['max'][key][i]))

            
            allvals = data['max'][key][i]
            arrvals = np.array(allvals) # Turn into an array for easy transposing

            hi = arrvals.transpose()
            hivals[j].append(np.mean(hi[0]))
            hiidx[j].append(np.mean(hi[1]))

            allvals = data['min'][key][i]
            arrvals = np.array(allvals) # Turn into an array for easy transposing

            lo = arrvals.transpose()
            lovals[j].append(np.mean(lo[0]))
            loidx[j].append(np.mean(lo[1]))

            
    #'''
    plt.subplot(4,1,1)
    plt.plot(hivals[0],'o',label='max - smooth') # Smooth
    plt.plot(hivals[1],'^',label='max - rough') # Smooth
    plt.legend()
    plt.xlim(-1,10)

    plt.subplot(4,1,2)
    plt.plot(hiidx[0],'o',label='max phase - smooth') # Smooth
    plt.plot(hiidx[1],'^',label='max phase - rough') # Smooth
    plt.legend()
    plt.xlim(-1,10)

    plt.subplot(4,1,3)
    plt.plot(lovals[0],'o',label='min - smooth') # Smooth
    plt.plot(lovals[1],'^',label='min - rough') # Smooth
    plt.legend()
    plt.xlim(-1,10)

    plt.subplot(4,1,4)
    plt.plot(loidx[0],'o',label='min phase - smooth') # Smooth
    plt.plot(loidx[1],'^',label='min phase - rough') # Smooth
    plt.legend()
    plt.xlim(-1,10)
    
    plt.tight_layout()
    
    plt.gcf().suptitle(label,fontsize=18)


    # Save the plot
    name = "plots_max_min_phase/%s.pdf" % (label)
    plt.savefig(name)
    #'''

    # Write out the values
    output  = "max:smooth,max:rough,maxphase:smooth,maxphase:rough,"
    output += "min:smooth,min:rough,minphase:smooth,minphase:rough,"
    output += "nstrides:smooth,nstrides:rough\n"
    for i in range(0,9):
        output += "%f,%f,%d,%d," % (hivals[0][i],hivals[1][i],hiidx[0][i],hiidx[1][i])
        output += "%f,%f,%d,%d,"  % (lovals[0][i],lovals[1][i],loidx[0][i],loidx[1][i])
        output += "%d,%d" % (nstrides[0][i],nstrides[1][i])
        
        output += "\n"

    name = "files_max_min_phase/%s.csv" % (label)
    outfile = open(name,'w')
    outfile.write(output)
    outfile.close()


In [ ]:
extrema['COM_vX']['max']['smooth'][0]

In [ ]:
keys = list(data.keys())

In [ ]:
tot = 0
for key in keys:
    if key[0:2]=="1_" and 'smooth' in key:
        print(key)
        tot += 1
print(tot)